<a href="https://colab.research.google.com/github/leonarddocchagas/data_science/blob/master/recife_bom_de_bola.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img width='30%' align='Center' src='https://github.com/leonarddocchagas/data_science/blob/master/projetos/recife_bom_de_bola/logo_recife_bom_de_bola_2014.png?raw=true' />

Maior campeonato de futebol de campos de várzea do mundo, o Recife Bom de Bola promove a inclusão social e a integração entre as diversas comunidades do Recife através do Esporte. Realizado pela Prefeitura do Recife, por meio da Secretaria de Esportes do Recife (SESP), o torneio envolve no ano de 2021, cinco categorias e mais de 350 times.

Além da interação social, o Recife Bom de Bola também vai garantir curso de arbitragem para representantes das comunidades e para professores vinculados à SESP. Ocorrerá ainda o Fórum Bom de Bola, para discussões diversas e benefícios para os destaques da categoria sub-15, que serão encaminhados para avaliações nos principais clubes do Recife.

<i>Fonte: https://recifebomdebola.com.br/o-projeto/<i/>

##Sobre os dados

Vamos trabalhar com 3 conjuntos de dados e todos vindo do http://dados.recife.pe.gov.br/.

* Dataset "campos": https://bit.ly/3n1kXg6
  * Dicionário: https://bit.ly/3kHPTiM
  * Variáveis:
    * rpa
    * endereco
    * bairro
    * latitude
    * longitude
    * privado
    * nome

* Dataset "Equipes": https://bit.ly/3gY6M7M
  * Dicionário: https://bit.ly/3gUhrAw
  * Variáveis:
    * rpa
    * categoria
    * equipe
    * cartao_amarelo
    * cartao_vermelho

* Dataset "Jogadores": https://bit.ly/3jFe0iF
  * Dicionário: https://bit.ly/3kHQBMY
  * Variáveis:
    * rpa_equipe
    * modalidade_equipe
    * nome_equipe
    * representante_equipe
    * tecnico
    * nome_jogador
    * numero_jogador
    * bairro_jogador
    * cidade_jogador
    * dtnascimento_jogador

##Obteção e transformação dos dados

**Carrega e verifica dataset**

In [26]:
# bibliotecas
import pandas as pd
import os
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from datetime import datetime

In [27]:
# import dataset equipes
df_equipes = 'https://raw.githubusercontent.com/leonarddocchagas/data_science/master/projetos/recife_bom_de_bola/equipes-participantes-recife-bom-de-bola.csv'
df_equipes = pd.read_csv(df_equipes, sep=';')
df_equipes.head()

,rpa,categoria,equipe,cartao_amarelo,cartao_vermelho
0,4,Sub 15,Projeto Show de Bola,0,0
1,4,Sub 17,Projeto Show de Bola,3,0
2,4,Aberto Masculino,show de bola jocum,3,1
3,2,Sub 15,Futebol E Recreação Santiago,7,0
4,2,Sub 17,Futebol E Recreação Santiago,5,4


In [28]:
# import dataset campos
df_campos = 'https://raw.githubusercontent.com/leonarddocchagas/data_science/master/projetos/recife_bom_de_bola/campos-recife-bom-de-bola.csv'
df_campos = pd.read_csv(df_campos, sep=';', error_bad_lines=False)
df_campos = df_campos.drop(columns='enderco')
df_campos.head()

,rpa,bairro,latitude,longitude,privado,nome
0,1,Santo Amaro,-8.040671,-34.878537,NÃO,CAMPO DO XI
1,1,Coque,-8.069009,-34.899431,NÃO,CAMPO DO BARRO
2,4,Torre,-8.045968,-34.913927,NÃO,CAMPO DO BUEIRÃO
3,1,Coque,-8.069379,-34.894672,SIM,CAMPO DO FÓRUM
4,2,Dois Unidos,-7.999053,-34.908048,SIM,CAMPO DO MINERVA


In [29]:
# import dataset jogadores
df_jogadores = 'https://raw.githubusercontent.com/leonarddocchagas/data_science/master/projetos/recife_bom_de_bola/dados-dos-jogadores.csv'
df_jogadores = pd.read_csv(df_jogadores, error_bad_lines=False, encoding='iso-8859-1', sep=';')
df_jogadores = df_jogadores.drop(columns=['REPRESENTANTE_EQUIPE', 'TECNICO'])
df_jogadores.head()

,RPA_EQUIPE,MODALIDADE_EQUIPE,NOME_EQUIPE,NOME_JOGADOR,NUMERO_JOGADOR,BAIRRO_JOGADOR,CIDADE_JOGADOR,DTNASCIMENTO_JOGADOR
0,4,Sub 15,Projeto Show de Bola,DEYVISON,3,DOIS IRMAOS,SAO LOURENCO DA MATA PE,1999-03-26
1,4,Sub 15,Projeto Show de Bola,DENYSON,3,DOIS IRMAOS,CAMARAGIBE,1999-01-10
2,4,Sub 15,Projeto Show de Bola,ALESANDRO,4,DOIS IRMAOS,CAMARAGIBE,1999-03-22
3,4,Sub 15,Projeto Show de Bola,MARCOS,5,DOIS IRMAOS,RECIFE PE,1999-03-16
4,4,Sub 15,Projeto Show de Bola,VITOR,6,DOIS IRMAOS,SAO LOURENCO DA MATA PE,2000-08-28


Identificamos que em nosso conjunto de dados **df_jogadores**, existem registros na coluna `DTNASCIMENTO_JOGADOR` que não segue o padrão. Vamos removelo para converter essa coluna num tipo date.

In [30]:
# identificando registros fora do padrão
df_jogadores['Teste'] = df_jogadores['DTNASCIMENTO_JOGADOR'].apply(lambda x: x.find('--') if 'tem' else 'nao')

In [31]:
# exibindo registros fora do padrão
df_jogadores.loc[df_jogadores['Teste'] != -1]


,RPA_EQUIPE,MODALIDADE_EQUIPE,NOME_EQUIPE,NOME_JOGADOR,NUMERO_JOGADOR,BAIRRO_JOGADOR,CIDADE_JOGADOR,DTNASCIMENTO_JOGADOR,Teste
265,5,Aberto Masculino,JUVENTUDE FUTEBOL CLUBE,emanoel,11,ESTANCIA,Recife,1-20-1--11,6
438,6,Aberto Masculino,os loucos club da vila,Anderson,7,IBURA,Recife,4-04-1--11,6
822,6,Aberto Masculino,CRUZEIRO FUTEBOL CLUB LAGOA ENCATADA,JOSÉ,9,COHAB,RECIFE,8-23-4--11,6
1364,3,Aberto Masculino,juventude futebol clube,EDSON,8,NaN,RECIFE,3-15-1--00,6
1489,4,Aberto Masculino,DE REPENTE,jesaias,5,NaN,recife,3-28-1--00,6
...,...,...,...,...,...,...,...,...,...
6700,2,Aberto Masculino,TENEBROSO F.C,PEDRO,12,NaN,NaN,3-28-1--00,6
6704,2,Aberto Masculino,TENEBROSO F.C,MARCIO,22,NaN,NaN,3-25-1--00,6
6719,2,Aberto Masculino,FAVELA FUTEBOL CLUB,réberte,17,NaN,NaN,4-03-1--00,6
6721,2,Aberto Masculino,FAVELA FUTEBOL CLUB,ROMÁRIO,18,NaN,NaN,4-03-1--00,6


Foram encontrados *168* fora do padrão. Vamos deletar e seguir com as análises.

In [32]:
# deletando registros fora do padrão de data
df_jogadores = df_jogadores.drop(df_jogadores.loc[df_jogadores['Teste'] != -1].index)
df_jogadores = df_jogadores.drop(columns='Teste')

# exibi dataset tratado
df_jogadores.head()

,RPA_EQUIPE,MODALIDADE_EQUIPE,NOME_EQUIPE,NOME_JOGADOR,NUMERO_JOGADOR,BAIRRO_JOGADOR,CIDADE_JOGADOR,DTNASCIMENTO_JOGADOR
0,4,Sub 15,Projeto Show de Bola,DEYVISON,3,DOIS IRMAOS,SAO LOURENCO DA MATA PE,1999-03-26
1,4,Sub 15,Projeto Show de Bola,DENYSON,3,DOIS IRMAOS,CAMARAGIBE,1999-01-10
2,4,Sub 15,Projeto Show de Bola,ALESANDRO,4,DOIS IRMAOS,CAMARAGIBE,1999-03-22
3,4,Sub 15,Projeto Show de Bola,MARCOS,5,DOIS IRMAOS,RECIFE PE,1999-03-16
4,4,Sub 15,Projeto Show de Bola,VITOR,6,DOIS IRMAOS,SAO LOURENCO DA MATA PE,2000-08-28


In [33]:
# tratando coluna nome_equipe
df_jogadores["NOME_EQUIPE"] = df_jogadores["NOME_EQUIPE"].str.title()

In [34]:
# convertindo coluna data para tipo date
df_jogadores['DTNASCIMENTO_JOGADOR'] = pd.to_datetime(df_jogadores['DTNASCIMENTO_JOGADOR'])

**Revisando os 3 datasets**

In [35]:
# dataset campo
df_campos.head()

,rpa,bairro,latitude,longitude,privado,nome
0,1,Santo Amaro,-8.040671,-34.878537,NÃO,CAMPO DO XI
1,1,Coque,-8.069009,-34.899431,NÃO,CAMPO DO BARRO
2,4,Torre,-8.045968,-34.913927,NÃO,CAMPO DO BUEIRÃO
3,1,Coque,-8.069379,-34.894672,SIM,CAMPO DO FÓRUM
4,2,Dois Unidos,-7.999053,-34.908048,SIM,CAMPO DO MINERVA


In [36]:
# dataset jogadores
df_jogadores.head()

,RPA_EQUIPE,MODALIDADE_EQUIPE,NOME_EQUIPE,NOME_JOGADOR,NUMERO_JOGADOR,BAIRRO_JOGADOR,CIDADE_JOGADOR,DTNASCIMENTO_JOGADOR
0,4,Sub 15,Projeto Show De Bola,DEYVISON,3,DOIS IRMAOS,SAO LOURENCO DA MATA PE,1999-03-26
1,4,Sub 15,Projeto Show De Bola,DENYSON,3,DOIS IRMAOS,CAMARAGIBE,1999-01-10
2,4,Sub 15,Projeto Show De Bola,ALESANDRO,4,DOIS IRMAOS,CAMARAGIBE,1999-03-22
3,4,Sub 15,Projeto Show De Bola,MARCOS,5,DOIS IRMAOS,RECIFE PE,1999-03-16
4,4,Sub 15,Projeto Show De Bola,VITOR,6,DOIS IRMAOS,SAO LOURENCO DA MATA PE,2000-08-28


In [37]:
# dataset equipes
df_equipes.head()

,rpa,categoria,equipe,cartao_amarelo,cartao_vermelho
0,4,Sub 15,Projeto Show de Bola,0,0
1,4,Sub 17,Projeto Show de Bola,3,0
2,4,Aberto Masculino,show de bola jocum,3,1
3,2,Sub 15,Futebol E Recreação Santiago,7,0
4,2,Sub 17,Futebol E Recreação Santiago,5,4


In [38]:
# df_equipes['equipe'] = df_equipes['equipe'].str.title()

## Visualizar 

In [39]:
# importa biblioteca para mapas
import folium
from folium import plugins

**Dataset: Campo**

In [40]:
# exibi as 5 primeiras entradas
df_campos.head()

,rpa,bairro,latitude,longitude,privado,nome
0,1,Santo Amaro,-8.040671,-34.878537,NÃO,CAMPO DO XI
1,1,Coque,-8.069009,-34.899431,NÃO,CAMPO DO BARRO
2,4,Torre,-8.045968,-34.913927,NÃO,CAMPO DO BUEIRÃO
3,1,Coque,-8.069379,-34.894672,SIM,CAMPO DO FÓRUM
4,2,Dois Unidos,-7.999053,-34.908048,SIM,CAMPO DO MINERVA


In [41]:
# Total de 22 campos
df_campos['nome'].count()

22

In [42]:
# eliminando espaços entre os nomes
df_campos['bairro'] = df_campos['bairro'].str.strip()

In [43]:
# total de campos por bairro
df_campos['bairro'].value_counts()

Várzea                2
Jiquiá                2
Coque                 2
Dois Unidos           2
Torre                 1
Brejo da Guabiraba    1
Mustardinha           1
Coqueiral             1
Campina do Barreto    1
Afogados              1
Cordeiro              1
Curado                1
Ipsep                 1
Macaxeira             1
Areias                1
Imbiribeira           1
Torräes               1
Santo Amaro           1
Name: bairro, dtype: int64

In [49]:
# plotando mapa
mapa = folium.Map(
    location = [-8.0704276,-34.894891],
    zoom_start=12,
    tiles='OpenStreetMap'
)

# criando pontos
for i, row in df_campos.iterrows(): 
  latitude = df_campos.at[i, 'latitude']
  longitude = df_campos.at[i, 'longitude']
  bairro = df_campos.at[i, 'bairro']

  folium.Marker(
      location = [latitude, longitude],
      popup=bairro,
      icon=folium.Icon('blue')
  ).add_to(mapa)

# exibe mapa
mapa

In [45]:
# plotando gráfico
coluna = "bairro"
grafico = px.bar(data_frame=df_campos, y=coluna,  orientation='h', barmode='group')
grafico.show()

**Dataset: Jogadores**

In [46]:
# dataset jogadores
df_jogadores.head()

,RPA_EQUIPE,MODALIDADE_EQUIPE,NOME_EQUIPE,NOME_JOGADOR,NUMERO_JOGADOR,BAIRRO_JOGADOR,CIDADE_JOGADOR,DTNASCIMENTO_JOGADOR
0,4,Sub 15,Projeto Show De Bola,DEYVISON,3,DOIS IRMAOS,SAO LOURENCO DA MATA PE,1999-03-26
1,4,Sub 15,Projeto Show De Bola,DENYSON,3,DOIS IRMAOS,CAMARAGIBE,1999-01-10
2,4,Sub 15,Projeto Show De Bola,ALESANDRO,4,DOIS IRMAOS,CAMARAGIBE,1999-03-22
3,4,Sub 15,Projeto Show De Bola,MARCOS,5,DOIS IRMAOS,RECIFE PE,1999-03-16
4,4,Sub 15,Projeto Show De Bola,VITOR,6,DOIS IRMAOS,SAO LOURENCO DA MATA PE,2000-08-28


In [47]:
# por faixa de idade
px.histogram(data_frame=df_jogadores,  x='DTNASCIMENTO_JOGADOR')

Uma das grandes vantagens de analisar pelo histograma é a facilidade de entender a distribuição dos dados. De forma simples, podemos ver que a grande maioria dos jogadores estão na faixa entre (1997-2005), mas também é possível encontrar registros abaixo de 1970. 

##Conclusão

​​O projeto Bom de Bola promovido pela prefeitura do Recife é interessante. Possibilidade, através do futebol, maior interação social. Os números de campos da cidade são bem diversificados.